Import libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV

%matplotlib inline

For now, limit the size of train/test data to 10000 rows

Read data

In [2]:
loadsize=10000
train = pd.read_csv('../Data/train.csv',nrows=loadsize,low_memory=False)
test = pd.read_csv('../Data/test.csv',nrows=loadsize,low_memory=False)

First take a look at the data.

In [3]:
train.head()

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0.0,0.0,False,False,...,98,98,998,999999998,998,998,9998,9998,BRANCH,1


In [4]:
print(train.columns)

Index(['ID', 'VAR_0001', 'VAR_0002', 'VAR_0003', 'VAR_0004', 'VAR_0005',
       'VAR_0006', 'VAR_0007', 'VAR_0008', 'VAR_0009',
       ...
       'VAR_1926', 'VAR_1927', 'VAR_1928', 'VAR_1929', 'VAR_1930', 'VAR_1931',
       'VAR_1932', 'VAR_1933', 'VAR_1934', 'target'],
      dtype='object', length=1934)


Two variables are missing from VAR_0001 to VAR_1934.

In [5]:
i = 1
MissingVar = []
for x in train.columns[1:-1]:
    var = 'VAR_'+'0'*(4-len(str(i)))+str(i)
    if x != var:
        MissingVar.append(var)
        i+=1
    i+=1
print('MissingVar: ',MissingVar)
del MissingVar

MissingVar:  ['VAR_0218', 'VAR_0240']


Now take a look at missing values. All together there are 1932 variables: VAR_0001 - VAR_1934 with VAR_0218 and VAR_0240 missing.

In [6]:
MissingVal = train[train.columns[1:-1]].isnull().sum().sort_values().apply(lambda x: np.round(x/train.shape[0],2))
print('Percentage of missing values:')
print(MissingVal)

Percentage of missing values:
VAR_0001    0.00
VAR_1298    0.00
VAR_1297    0.00
VAR_1296    0.00
VAR_1295    0.00
VAR_1294    0.00
VAR_1293    0.00
VAR_1292    0.00
VAR_1291    0.00
VAR_1290    0.00
VAR_1289    0.00
VAR_1288    0.00
VAR_1287    0.00
VAR_1299    0.00
VAR_1286    0.00
VAR_1284    0.00
VAR_1283    0.00
VAR_1282    0.00
VAR_1281    0.00
VAR_1280    0.00
VAR_1279    0.00
VAR_1278    0.00
VAR_1277    0.00
VAR_1276    0.00
VAR_1275    0.00
VAR_1274    0.00
VAR_1273    0.00
VAR_1285    0.00
VAR_1272    0.00
VAR_1300    0.00
            ... 
VAR_0356    0.01
VAR_0353    0.01
VAR_0355    0.01
VAR_0354    0.01
VAR_0350    0.01
VAR_0212    0.09
VAR_0073    0.70
VAR_0074    0.70
VAR_0211    0.87
VAR_0210    0.87
VAR_0208    0.87
VAR_0176    0.88
VAR_0179    0.88
VAR_0166    0.90
VAR_0169    0.90
VAR_0178    0.92
VAR_0168    0.93
VAR_0209    0.94
VAR_0159    0.96
VAR_0156    0.96
VAR_0177    0.98
VAR_0167    0.98
VAR_0205    0.99
VAR_0158    0.99
VAR_0206    0.99
VAR_0157    0.99
V

Delete variables with missing values over 50%

In [7]:
Ind = MissingVal[MissingVal>0.5].index
train = train.drop(Ind,axis=1)
test = test.drop(Ind,axis=1)
del MissingVal,Ind

Now try to figure out categorical variables.

In [8]:
Features = train[train.columns[1:-1]]

In [9]:
Features.dtypes.value_counts()

int64      1404
float64     467
object       37
dtype: int64

Now try to figure out the number of categorical variables for numerical and string variables.

In [10]:
NumVar,NumVarIndex,ObjVar,ObjVarIndex = [],[],[],[]
for x in Features.columns[Features.dtypes!='object']:
    NumVar.append(len(Features[x].value_counts()))
    NumVarIndex.append(x)
for x in Features.columns[Features.dtypes=='object']:
    ObjVar.append(len(Features[x].value_counts()))
    ObjVarIndex.append(x)
NumVar = pd.Series(NumVar,index=NumVarIndex).sort_values()
ObjVar = pd.Series(ObjVar,index=ObjVarIndex).sort_values()
del NumVarIndex,ObjVarIndex,Features
print('Numerical variable value counts: \n',NumVar)
print('String variable value counts: \n',ObjVar)

Numerical variable value counts: 
 VAR_0529       1
VAR_0041       1
VAR_0042       1
VAR_0189       1
VAR_0446       1
VAR_0246       1
VAR_1428       1
VAR_0847       1
VAR_0526       1
VAR_0527       1
VAR_0528       1
VAR_0223       1
VAR_0221       1
VAR_0215       1
VAR_0530       1
VAR_0203       1
VAR_0394       1
VAR_0395       1
VAR_0199       1
VAR_0197       1
VAR_0396       1
VAR_0397       1
VAR_0398       1
VAR_0193       1
VAR_0192       1
VAR_0399       1
VAR_0191       1
VAR_0190       1
VAR_0040       1
VAR_0039       1
            ... 
VAR_1801    5379
VAR_1372    5441
VAR_1802    5599
VAR_0241    5677
VAR_0652    5969
VAR_0951    6116
VAR_0950    6195
VAR_0970    6242
VAR_0649    6350
VAR_1496    6371
VAR_1497    6464
VAR_0648    6661
VAR_1494    6818
VAR_1201    6824
VAR_1489    6902
VAR_0887    7236
VAR_0704    7382
VAR_1495    7933
VAR_1087    8311
VAR_0543    8410
VAR_0541    8494
VAR_0899    8751
VAR_1181    8857
VAR_1179    8904
VAR_1180    8939
VAR_1081    8

Delete variables with only 1 value

In [11]:
Ind = NumVar[:len(NumVar[NumVar==1])].index
train = train.drop(Ind,axis=1)
test = test.drop(Ind,axis=1)
Ind = ObjVar[:len(ObjVar[ObjVar==1])].index
train = train.drop(Ind,axis=1)
test = test.drop(Ind,axis=1)
del Ind
NumVar = NumVar[len(NumVar[NumVar==1]):]
ObjVar = ObjVar[len(ObjVar[ObjVar==1]):]

In [12]:
print('Now the total number of the variables becomes %s with %s numerical variables and %s string variables.'%(len(NumVar)+len(ObjVar),len(NumVar),len(ObjVar)))
print('Numerical variable value counts: \n',NumVar)
print('String variable value counts: \n',ObjVar)

Now the total number of the variables becomes 1845 with 1821 numerical variables and 24 string variables.
Numerical variable value counts: 
 VAR_0180       2
VAR_0116       2
VAR_0740       2
VAR_0739       2
VAR_0737       2
VAR_0139       2
VAR_0459       2
VAR_0138       2
VAR_0736       2
VAR_0182       2
VAR_0733       2
VAR_0130       2
VAR_0732       2
VAR_0449       2
VAR_0567       2
VAR_0566       2
VAR_0563       2
VAR_0181       2
VAR_1427       2
VAR_0122       2
VAR_0445       2
VAR_0383       2
VAR_0195       2
VAR_0476       2
VAR_0471       2
VAR_0490       2
VAR_0470       2
VAR_0277       2
VAR_0276       2
VAR_0275       2
            ... 
VAR_1801    5379
VAR_1372    5441
VAR_1802    5599
VAR_0241    5677
VAR_0652    5969
VAR_0951    6116
VAR_0950    6195
VAR_0970    6242
VAR_0649    6350
VAR_1496    6371
VAR_1497    6464
VAR_0648    6661
VAR_1494    6818
VAR_1201    6824
VAR_1489    6902
VAR_0887    7236
VAR_0704    7382
VAR_1495    7933
VAR_1087    8311
VAR_0543 

Missing values again. Fill in the missing values with the mode value for each variable

In [13]:
MissingValTrain = train[train.columns[1:-1]].isnull().sum().sort_values()
MissingValTest = test[test.columns[1:]].isnull().sum().sort_values()
MissingValTrain = MissingValTrain[MissingValTrain>0].index
MissingValTest = MissingValTest[MissingValTest>0].index
print('Variables with missing values for train data: \n',MissingValTrain)
print('Variables with missing values for test data: \n',MissingValTest)

Variables with missing values for train data: 
 Index(['VAR_0086', 'VAR_0089', 'VAR_0088', 'VAR_0200', 'VAR_0087', 'VAR_0198',
       'VAR_0085', 'VAR_0080', 'VAR_0082', 'VAR_0081',
       ...
       'VAR_0369', 'VAR_0370', 'VAR_0371', 'VAR_0372', 'VAR_0373', 'VAR_0374',
       'VAR_0375', 'VAR_0376', 'VAR_0350', 'VAR_0212'],
      dtype='object', length=440)
Variables with missing values for test data: 
 Index(['VAR_0086', 'VAR_0089', 'VAR_0088', 'VAR_0200', 'VAR_0087', 'VAR_0198',
       'VAR_0085', 'VAR_0080', 'VAR_0082', 'VAR_0081',
       ...
       'VAR_0369', 'VAR_0370', 'VAR_0371', 'VAR_0372', 'VAR_0373', 'VAR_0374',
       'VAR_0375', 'VAR_0376', 'VAR_0350', 'VAR_0212'],
      dtype='object', length=440)


In [14]:
for i in MissingValTrain:
    Ind = train[i][train[i].isnull()].index
    train.loc[Ind,i]=train[i].mode()[0]
for i in MissingValTest:
    Ind = test[i][test[i].isnull()].index
    test.loc[Ind,i]=test[i].mode()[0]

Now manually process string variables.

String variable value counts:

VAR_0236       2

VAR_0232       2

VAR_0466       2

VAR_0230       2

VAR_0226       2

VAR_0001       3

VAR_0467       4

VAR_0005       4

VAR_0354       4

VAR_0353       4

VAR_0352       4

VAR_1934       5

VAR_0283       7

VAR_0305       7

VAR_0325       9

VAR_0237      42

VAR_0342      47

VAR_0274      56

VAR_0493     167

VAR_0404     230

VAR_0217     395

VAR_0204    1188

VAR_0075    1447

VAR_0200    3473

1. For variables with only 2 different values, directly apply LabelEncoding.
2. For variables with >2 but <=5 different values, directly apply OneHotEncoding and drop the original variables.
3. For VAR_0283, VAR_0305 and VAR_0325, some values only contain very few entries, categorize them as "other" and apply OneHotEncoding.
4. VAR_0237, VAR_0342, VAR_0274 and VAR_0200 are state and names. By intuition we convert these string values to the frequency of each entry to represent the popularity of the service in that area.
5. VAR_0217, VAR_0204 and VAR_0075 are datetime type variables, convert them to numerical variables.
6. For VAR_0493 and VAR_0404, there are more than 600 values but most of the them take -1 (>90%), so categorize the rest as "other".


1. For variables with only 2 different values, directly apply LabelEncoding inline.

In [15]:
for i in ObjVar[ObjVar==2].index:
    le=LabelEncoder()
    le.fit(train[i])
    train[i]=le.transform(list(train[i]))
    test[i]=le.transform(list(test[i]))

2. For variables with >2 but <=5 different values, directly apply OneHotEncoding and drop the original variables.

In [16]:
for i in set(ObjVar[ObjVar<=5].index)-set(ObjVar[ObjVar==2].index):
    le = LabelEncoder()
    oh = OneHotEncoder()
    le.fit(train[i])
    train[i] = le.transform(list(train[i]))
    test[i] = le.transform(list(test[i]))
    temp = np.reshape(list(train[i]),(len(train[i]),-1))
    oh.fit(temp)
    temp = pd.DataFrame(oh.transform(temp).toarray(),columns=[i+'_%s'%(x) for x in range(ObjVar[i])])
    train = pd.concat([train,temp],axis=1).drop(i,axis=1)
    temp = np.reshape(list(test[i]),(len(test[i]),-1))
    temp = pd.DataFrame(oh.transform(temp).toarray(),columns=[i+'_%s'%(x) for x in range(ObjVar[i])])
    test = pd.concat([test,temp],axis=1).drop(i,axis=1)
    del le, oh, temp

3. For VAR_0283, VAR_0305 and VAR_0325, some values only contain very few entries (<1% of loadsize), categorize them as "other" and apply OneHotEncoding.

In [17]:
# Re-categorize data
for i in set(ObjVar[ObjVar<10].index)-set(ObjVar[ObjVar<=5].index):
    valcount = train[i].value_counts()
    for j in valcount.index:
        if valcount[j]<loadsize/100:
            Ind = train[i][train[i]==j].index
            train.loc[Ind,i] = 'Other'
    valcount = test[i].value_counts()
    for j in valcount.index:
        if valcount[j]<loadsize/100:
            Ind = test[i][test[i]==j].index
            test.loc[Ind,i] = 'Other'
    del valcount, Ind

In [18]:
# Now apply OneHotEncoding
for i in set(ObjVar[ObjVar<10].index)-set(ObjVar[ObjVar<=5].index):
    le = LabelEncoder()
    oh = OneHotEncoder()
    le.fit(train[i])
    train[i] = le.transform(list(train[i]))
    test[i] = le.transform(list(test[i]))
    temp = np.reshape(list(train[i]),(len(train[i]),-1))
    oh.fit(temp)
    temp = pd.DataFrame(oh.transform(temp).toarray(),columns=[i+'_%s'%(x) for x in range(len(train[i].value_counts()))])
    train = pd.concat([train,temp],axis=1).drop(i,axis=1)
    temp = np.reshape(list(test[i]),(len(test[i]),-1))
    temp = pd.DataFrame(oh.transform(temp).toarray(),columns=[i+'_%s'%(x) for x in range(len(test[i].value_counts()))])
    test = pd.concat([test,temp],axis=1).drop(i,axis=1)
    del le, oh, temp

4. VAR_0237, VAR_0342, VAR_0274 and VAR_0200 are state and names. By intuition we convert these string values to the frequency of each entry to represent the popularity of the service in that area.

In [19]:
Features = ['VAR_0237', 'VAR_0342', 'VAR_0274', 'VAR_0200']
for i in Features:
    valcount = train[i].value_counts()
    for j in valcount.index:
        Ind = train[i][train[i] == j].index
        train.loc[Ind,i] = valcount[j]
    valcount = test[i].value_counts()
    for j in valcount.index:
        Ind = test[i][test[i] == j].index
        test.loc[Ind,i] = valcount[j]
    del valcount, Ind
del Features

5. VAR_0217, VAR_0204 and VAR_0075 are datetime type variables, convert them to numerical variables.

In [20]:
Features = ['VAR_0217', 'VAR_0204', 'VAR_0075']
Month = {'JAN':'01','FEB':'02','MAR':'03','APR':'04','MAY':'05','JUN':'06','JUL':'07','AUG':'08','SEP':'09','OCT':'10','NOV':'11','DEC':'12'}

for i in Features:
    train[i] = train[i].apply(lambda x: int('20'+x[5:7]+Month[x[2:5]]+x[:2]+x[8:10]+x[11:13]+x[14:]))
    test[i] = test[i].apply(lambda x: int('20'+x[5:7]+Month[x[2:5]]+x[:2]+x[8:10]+x[11:13]+x[14:]))

del Features, Month

6. For VAR_0493 and VAR_0404, there are more than 600 values but most of the them take -1 (>90%), so categorize the rest as "other".

In [21]:
Features = ['VAR_0493', 'VAR_0404']

for i in Features:
    Ind = train[i][train[i]!='-1'].index
    train.loc[Ind,i] = 0
    Ind = train[i][train[i]=='-1'].index
    train.loc[Ind,i] = -1
    Ind = test[i][test[i]!='-1'].index
    test.loc[Ind,i] = 0
    Ind = test[i][test[i]=='-1'].index
    test.loc[Ind,i] = -1
del Ind

Now all the missing values have been filled and all the variables have been converted to numerical variables, we are ready to train our model. 

Use GridSearchCV to find the best parameters

In [ ]:
'''
print('Begin training the model...')
ID = test.ID

parameters = {
        'max_depth': [3,4],
        'learning_rate': [0.05,0.10,0.15],
        'n_estimators': [50,100]
    }

model = xgb.XGBClassifier(
    max_depth=3, 
    learning_rate=0.05,
    objective='binary:logistic', 
    eval_metric='auc',
    n_estimators=100
)

gsearch = GridSearchCV(model, param_grid=parameters, scoring='roc_auc', cv=3)
gsearch.fit(train.drop(['ID','target'],axis=1),train.target)
print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
'''

In [24]:
model = xgb.XGBClassifier(
    max_depth=3, 
    learning_rate=0.10,
    objective='binary:logistic', 
    eval_metric='auc',
    n_estimators=100
)
cross_val_score(model,train.drop(['ID','target'],axis=1),train.target,scoring='roc_auc',cv=5)

array([0.74667317, 0.76031469, 0.74700903, 0.75872523, 0.75594737])

In [25]:
model = xgb.XGBClassifier(
        max_depth=3, 
        learning_rate=0.10,
        objective='binary:logistic', 
        eval_metric='auc',
        n_estimators=100
    )


In [26]:
model.fit(train.drop(['ID','target'],axis=1), train.target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)